<a href="https://colab.research.google.com/github/ViniciusSodre07/NLP/blob/main/scraping_music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.IMPORTANDO AS BIBLIOTECAS**

---

In [ ]:
#IMPORTANDO AS BIBLIOTECAS
import requests
from bs4 import BeautifulSoup
import pandas as pd

# **2.DATASET MÚSICAS**

---

In [ ]:
#Importando do site valume as músicas do artista Tim Maia

page = requests.get("https://www.vagalume.com.br/tim-maia/")
soup = BeautifulSoup(page.content, 'html.parser')
lista_alfabetica = BeautifulSoup(str(soup.findChildren(id = "alfabetMusicList")), 'html.parser')
a_tag = lista_alfabetica.findAll('a')

musicas = []
for a in a_tag:
    nome_musica = a.text
    if not(nome_musica == 'TRADUÇÃO' or nome_musica == ''):
        link_musica = a['href']
        musicas.append([nome_musica, link_musica])

for i in range(len(musicas)):
    link = "https://www.vagalume.com.br" + str(musicas[i][1])
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    h3_tag = soup.findAll('h3')
    if len(h3_tag) != 0:
        album = h3_tag[0].text
    else:
        album = ''
    lyrics = soup.findChildren(id = 'lyrics')
    lyrics = str(lyrics[0])
    lyrics = lyrics.replace('<div id="lyrics">', '')
    lyrics = lyrics.replace('<div data-plugin="googleTranslate" id="lyrics">', '')
    lyrics = lyrics.replace('<br/>', ' ')
    lyrics = lyrics.replace("\'","'")
    lyrics = lyrics.replace('</div>', '')
    musicas[i].append(album)
    musicas[i].append(lyrics)

musicas = pd.DataFrame(musicas, columns=['Nome da Música', 'link', 'album', 'letra'])
musicas.to_csv('musicas.csv', index=False)

In [ ]:
# Ler o arquivo CSV 'musicas.csv'
musicas = pd.read_csv('musicas.csv')

# Exibir as primeiras 20 linhas do DataFrame #https://www.vagalume.com.br/tim-maia/amores.html
musicas.head(20)


In [ ]:
# Obter e Concatenar o número de linhas com a string corretamente
print('Número de linhas: ' + str(musicas.shape[0]))

Número de linhas: 266


# **3.TRATANDO A BASE DE DADOS**

---



In [ ]:
# Definir o padrão de código para músicas instrumentais (convertido para minúsculas)
instrumental_code = 'instrumental<img alt="instrumental" class="instrumental-icon" src="/img/etc/instrumental.png"/>'


# Filtrar e remover as linhas onde a coluna "letra" contenha o código de instrumental, ignorando maiúsculas e minúsculas
musicas_filtradas = musicas[~musicas['letra'].str.lower().str.contains(instrumental_code)]


# Salvar o DataFrame limpo em um novo arquivo CSV
musicas_filtradas.to_csv('musicas_filtradas.csv', index=False)


In [ ]:
musicas_filtradas.head(20)

In [ ]:
# Obter e Concatenar o número de linhas com a string corretamente
print('Número de linhas: ' + str(musicas_filtradas.shape[0]))

# **4.FREQUENCIA DE PALAVRAS**

---



In [ ]:
pip install spacy

In [ ]:
python -m spacy download pt_core_news_sm

In [ ]:

import spacy
from collections import Counter
import matplotlib.pyplot as plt

# Carregar o modelo spaCy para português
nlp = spacy.load('pt_core_news_sm')


# Função para processar e contar a frequência de palavras
def contar_frequencia(texto):
    # Processa o texto usando spaCy
    doc = nlp(texto)
    # Cria uma lista de palavras, removendo stopwords e pontuações
    palavras = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
    return palavras

# Aplicar a função para contar a frequência em todas as letras
musicas_filtradas['palavras'] = musicas_filtradas['letra'].apply(contar_frequencia)

# Concatenar todas as listas de palavras em uma única lista
todas_as_palavras = [palavra for sublist in musicas_filtradas['palavras'] for palavra in sublist]

# Contar a frequência das palavras
frequencia_palavras = Counter(todas_as_palavras)

# Exibir as 10 palavras mais comuns
palavras_comuns = frequencia_palavras.most_common(10)
print("10 Palavras Mais Comuns:")
for palavra, contagem in palavras_comuns:
    print(f"{palavra}: {contagem}")

# Visualização das palavras mais comuns
labels, values = zip(*palavras_comuns)
plt.figure(figsize=(10, 5))
plt.bar(labels, values)
plt.title('10 Palavras Mais Comuns nas Músicas de Tim Maia')
plt.xlabel('Palavras')
plt.ylabel('Frequência')
plt.xticks(rotation=45)
plt.show()


SyntaxError: invalid syntax (<ipython-input-65-605c13190dd9>, line 1)